In [ ]:
# AbacusResearch/RasGulla1-7b

In [ ]:
# !pip install -qU transformers datasets loralib einops xformers

In [ ]:
# !pip install git+https://github.com/huggingface/accelerate.git

In [ ]:
# !pip install -qqq torch

In [ ]:
# !pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
# !pip install peft

Installing torch and unsloth had been a nightmare since the torch version on colab breaks unsloth for some reason, following code taken from unsloth's huggingface account https://huggingface.co/datasets/unsloth/notebooks/blob/main/DPO_Zephyr_Unsloth_Example.ipynb

In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

# DO NOT RUN, TO CHECK THE PREVIOUS ISSUE WITH TORCH

In [ ]:
# this is old
import torch
print(torch.__version__)

2.1.0+cu121


# After unsloth's notebook torch, run from here

In [3]:
print(torch.__version__) # 2.2.1 is required for unsloth

2.2.1+cu121


In [4]:
major_version, minor_version = torch.cuda.get_device_capability()
print(f"CUDA major version: {major_version}, minor version: {minor_version}")

CUDA major version: 8, minor version: 0


In [5]:
from unsloth import FastLanguageModel # this returns both model and tokenizer

# Using transformers, will switch to unsloth

In [ ]:
# quantization
# quiantization_config = BitsAndBytesConfig(load_in_4bit=True)

In [ ]:
# tokenizer
# tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

In [ ]:
# load model with 4bit quantization with the config
# model = AutoModelForCausalLM.from_pretrained("google/gemma-7b", quantization_config=quiantization_config)

# unsloth

In [6]:
# "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-7b",
    max_seq_length = 2048, # use RoPE anyway so does not matter
    dtype = None,# float16 for low gpu, V100, bfloat16 for Ampere+, detects automatically
    load_in_4bit = True
)

Unsloth: You passed in `google/gemma-7b` and `load_in_4bit = True`.
We shall load `unsloth/gemma-7b-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [8]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear4bit(in_features=24576, out_features=3072, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )

In [9]:
# applying lora on specified weight metrices
# w and k used on paper
# here applying on all weight metrices
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], # weights modues to apply matrix decomposition
    lora_alpha = 32, # for learning rate i suppose?
    lora_dropout = 0, # 0 is optimized
    bias = "none",    # "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # rank stabilized
    loftq_config = None
)

Unsloth 2024.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Custom datset

In [10]:
import pandas as pd
df_old = pd.read_csv("div_move_exp_5000.csv")
df_custom = pd.read_csv("custom_df.csv")
df_custom.head()

,Moves,Explanation
0,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3,"Black challenges the center directly with d5, ..."
1,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6,"Black develops the knight to c6, adding pressu..."
2,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6 Nc3,White counters by developing another knight to...
3,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6 Nc3 Bg4,"Black pins the knight on f3 with Bg4, creating..."
4,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6 Nc3 Bg4 Be2,"White prepares to break the pin with Be2, also..."


In [11]:
df_custom.shape

(669, 2)

In [13]:
df_custom[["prev_moves", "last_move"]] = df_custom["Moves"].str.rsplit(n=1, expand=True)
df_custom.drop(columns=["Moves"], inplace=True)
df_custom.head()

,Explanation,prev_moves,last_move
0,"Black challenges the center directly with d5, ...",d4 Nf6 c4 c5 e3 cxd4 exd4 d5,Nf3
1,"Black develops the knight to c6, adding pressu...",d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3,Nc6
2,White counters by developing another knight to...,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6,Nc3
3,"Black pins the knight on f3 with Bg4, creating...",d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6 Nc3,Bg4
4,"White prepares to break the pin with Be2, also...",d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6 Nc3 Bg4,Be2


In [14]:
df_custom["last_move_fill"] = df_custom.apply(lambda row : row["prev_moves"] if row["last_move"] == None else row["last_move"], axis = 1)
df_custom.drop(columns=["last_move"],inplace=True)
df_custom.head()

,Explanation,prev_moves,last_move_fill
0,"Black challenges the center directly with d5, ...",d4 Nf6 c4 c5 e3 cxd4 exd4 d5,Nf3
1,"Black develops the knight to c6, adding pressu...",d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3,Nc6
2,White counters by developing another knight to...,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6,Nc3
3,"Black pins the knight on f3 with Bg4, creating...",d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6 Nc3,Bg4
4,"White prepares to break the pin with Be2, also...",d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6 Nc3 Bg4,Be2


In [16]:
df_custom.rename(columns={"Explanation":"explanation"}, inplace=True)
df_custom = df_custom[["prev_moves", "last_move_fill", "explanation"]]
df_custom.head()

,prev_moves,last_move_fill,explanation
0,d4 Nf6 c4 c5 e3 cxd4 exd4 d5,Nf3,"Black challenges the center directly with d5, ..."
1,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3,Nc6,"Black develops the knight to c6, adding pressu..."
2,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6,Nc3,White counters by developing another knight to...
3,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6 Nc3,Bg4,"Black pins the knight on f3 with Bg4, creating..."
4,d4 Nf6 c4 c5 e3 cxd4 exd4 d5 Nf3 Nc6 Nc3 Bg4,Be2,"White prepares to break the pin with Be2, also..."


# Generated Dataset

In [12]:
df_old.head()

,Unnamed: 0.1,Unnamed: 0,prev_moves,last_move_fill,explanation
0,0,0,d4,d4,The last move of playing d4 again likely aims ...
1,1,1,d4,d5,The move d5 by black is a common response to w...
2,2,2,d4 d5,c4,"The move c4, also known as the Queen's Gambit,..."
3,3,3,d4 d5 c4,c6,The move c6 is a common response to the openin...
4,4,4,d4 d5 c4 c6,cxd5,"The last move, cxd5, is logical because it cap..."


In [17]:
df_old = df_old[["prev_moves", "last_move_fill", "explanation"]]
df_old.head()

,prev_moves,last_move_fill,explanation
0,d4,d4,The last move of playing d4 again likely aims ...
1,d4,d5,The move d5 by black is a common response to w...
2,d4 d5,c4,"The move c4, also known as the Queen's Gambit,..."
3,d4 d5 c4,c6,The move c6 is a common response to the openin...
4,d4 d5 c4 c6,cxd5,"The last move, cxd5, is logical because it cap..."


In [24]:
dataset = pd.concat([df_old, df_custom])
dataset = dataset.reset_index(drop=True)
dataset.shape

(5669, 3)

In [25]:
dataset.to_csv("dataset.csv", index=False)

# Loading dataset

In [26]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="dataset.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prev_moves', 'last_move_fill', 'explanation'],
        num_rows: 5669
    })
})

In [5]:
prompt_chess = """
Instruction:{}; previous moves:{}; last move:{}.
Response:{}"""

In [29]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func_chess(examples):
    instruction = "In a paragraph, explain the rationale behind the last move, where all previous moves are:"
    prevs       = examples["prev_moves"]
    lasts       = examples["last_move_fill"]
    exps     = examples["explanation"]
    texts = []
    for prev, last, exp in zip(prevs, lasts, exps):
        text = prompt_chess.format(instruction, prev, last, exp) + EOS_TOKEN
        #print(text)
        texts.append(text)
    return {"text" : texts}

In [30]:
dataset = dataset.map(formatting_prompts_func_chess, batched = True)

Map:   0%|          | 0/5669 [00:00<?, ? examples/s]

In [31]:
dataset["train"][1]

{'prev_moves': 'd4',
 'last_move_fill': 'd5',
 'explanation': "The move d5 by black is a common response to white's move d4 in chess. By playing d5, black aims to control the center of the board and challenge white's dominance in that area. This move also opens up lines for the black queen and bishop to potentially develop. Additionally, by advancing the pawn to d5, black prepares to potentially push it further to d4 to create pressure on white's pieces and limit their mobility. Overall, playing d5 is a strategic move that aims to equalize the game and create additional options for black's pieces to develop.",
 'text': "\nInstruction:In a paragraph, explain the rationale behind the last move, where all previous moves are:; previous moves:d4; last move:d5. \nResponse:The move d5 by black is a common response to white's move d4 in chess. By playing d5, black aims to control the center of the board and challenge white's dominance in that area. This move also opens up lines for the black q

In [33]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [34]:
dataset["train"]["text"][0]

"\nInstruction:In a paragraph, explain the rationale behind the last move, where all previous moves are:; previous moves:d4; last move:d4. \nResponse:The last move of playing d4 again likely aims to continue controlling the center of the board and establishing a strong pawn presence. By playing d4 for a second time, the player may be reinforcing their control over the central squares and preparing to develop their pieces for a more aggressive and strategic play. Additionally, repeating the move d4 could potentially be a part of a specific opening or strategy that the player is following, such as the Queen's Gambit or the King's Indian Defense. Overall, playing d4 again maintains a solid foundation for future moves and sets the stage for a dynamic and flexible game plan.<eos>"

In [35]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences, good for flash attention i think.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 1000,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(), # trying to use bf6
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs"
    )
)

Map (num_proc=2):   0%|          | 0/5669 [00:00<?, ? examples/s]

In [36]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,669 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 708
 "-____-"     Number of trainable parameters = 50,003,968


Step,Training Loss
50,1.173300
100,0.807900
150,0.650800
200,0.608400
250,0.544600
300,0.537500
350,0.520500
400,0.502000
450,0.476800
500,0.485200


In [37]:
trainer_stats

TrainOutput(global_step=708, training_loss=0.578361576223104, metrics={'train_runtime': 1875.456, 'train_samples_per_second': 3.023, 'train_steps_per_second': 0.378, 'total_flos': 7.796817484282675e+16, 'train_loss': 0.578361576223104, 'epoch': 1.0})

In [38]:
model.save_pretrained("trained-model")

In [40]:
from huggingface_hub import notebook_login
notebook_login()

In [41]:
gemma_model = "adnaan525/gemma_7B_unsloth_custom_old"

model.push_to_hub(
    gemma_model, use_auth_token=True
)

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/200M [00:00<?, ?B/s]

Saved model to https://huggingface.co/adnaan525/gemma_7B_unsloth_custom_old


## Loading the saved trained model

In [4]:
import torch
from unsloth import FastLanguageModel # gpu out of memory, restart session
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "trained-model",
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: Tesla V100-SXM2-16GB. Max memory: 15.773 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
inputs = tokenizer(
[
    prompt_chess.format(
        "in a paragraph, explain the rationale behind the last move, where all previous moves are",
        "e4 c5 Nf3 Nc6 Bc4 g6 Ng5 Ne5 Bb3 h6 Nf3 Bg7 Nxe5 Bxe5 O-O Qc7 Qf3 Nf6 h3 h5 c3 O-O d3 Nh7 g4 hxg4 Qxg4 d6 Qxg6+ Kh8 Bxf7 Bxh3 Re1 Rg8 Bxg8 Rxg8 Qxg8+ Kxg8 Re3",
        "Qc8",
        ""
    )
], return_tensors = "pt").to("cuda")

In [20]:
import textwrap
def process_response(response):
    lines = response.replace("<bos>", "").replace("<eos>", "").split("\n")
    wrapped_lines = [textwrap.fill(line, width=100) for line in lines]
    for wrapped_line in wrapped_lines:
        print(wrapped_line)

In [21]:
%%time
outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
process_response(tokenizer.decode(outputs[0]))


Instruction:in a paragraph, explain the rationale behind the last move, where all previous moves
are; previous moves:e4 c5 Nf3 Nc6 Bc4 g6 Ng5 Ne5 Bb3 h6 Nf3 Bg7 Nxe5 Bxe5 O-O Qc7 Qf3 Nf6 h3 h5 c3
O-O d3 Nh7 g4 hxg4 Qxg4 d6 Qxg6+ Kh8 Bxf7 Bxh3 Re1 Rg8 Bxg8 Rxg8 Qxg8+ Kxg8 Re3; last move:Qc8.
Response:The last move, Qc8, was likely made to protect the c7 pawn and potentially prepare for a
future attack on the opponent's king. By moving the queen to c8, the player is also freeing up the
c7 square for their rook to potentially come into play. Additionally, the queen on c8 is well-placed
to support any future pawn advances or piece maneuvers in the center of the board. Overall, the move
Qc8 is a strategic decision to improve the player's position and maintain control over key squares
on the board.
CPU times: user 6.85 s, sys: 8.5 ms, total: 6.86 s
Wall time: 6.85 s


In [22]:
# before custom
process_response('''Instruction:in a paragraph, explain the rationale behind the last move, where all previous moves are; previous moves:e4 c5 Nf3 Nc6 Bc4 g6 Ng5 Ne5 Bb3 h6 Nf3 Bg7 Nxe5 Bxe5 O-O Qc7 Qf3 Nf6 h3 h5 c3 O-O d3 Nh7 g4 hxg4 Qxg4 d6 Qxg6+ Kh8 Bxf7 Bxh3 Re1 Rg8 Bxg8 Rxg8 Qxg8+ Kxg8 Re3; last move:Qc8. \nResponse:The last move, Qc8, was likely made to protect the bishop on h3 and prevent it from being captured by the opponent's queen. By moving the queen to c8, the player is also preparing to potentially launch an attack on the opponent's king side. Additionally, the queen on c8 is now in a more centralized position, allowing for better control of the board and potential threats to the opponent's pieces. Overall, the move Qc8 was a strategic decision to improve the player's position and maintain control of the game.<eos>
''')

Instruction:in a paragraph, explain the rationale behind the last move, where all previous moves
are; previous moves:e4 c5 Nf3 Nc6 Bc4 g6 Ng5 Ne5 Bb3 h6 Nf3 Bg7 Nxe5 Bxe5 O-O Qc7 Qf3 Nf6 h3 h5 c3
O-O d3 Nh7 g4 hxg4 Qxg4 d6 Qxg6+ Kh8 Bxf7 Bxh3 Re1 Rg8 Bxg8 Rxg8 Qxg8+ Kxg8 Re3; last move:Qc8.
Response:The last move, Qc8, was likely made to protect the bishop on h3 and prevent it from being
captured by the opponent's queen. By moving the queen to c8, the player is also preparing to
potentially launch an attack on the opponent's king side. Additionally, the queen on c8 is now in a
more centralized position, allowing for better control of the board and potential threats to the
opponent's pieces. Overall, the move Qc8 was a strategic decision to improve the player's position
and maintain control of the game.

